In [1]:
# !pip install pyreadstat

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   --- ------------------------------------ 0.2/2.4 MB 3.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.4 MB 6.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.4 MB 8.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.4 MB 7.9 MB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.4 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 8.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/425.0 kB ? eta -:--:--
   --------------------------------------  419.8/425.0 kB 13.2 MB/s eta 0:00:01
   ---------------------------------------- 425.0/425.0 kB 8.8 MB/s eta 0:00:00


## START

I saved all the 'cleaned' up datasets as a csv to the folder clean_data for ease of viewing.

Note: For years 1985 1986 and 1988 the data explaining each variable is somewhat corrupted, although historically variable numbers are consistent, so we could use the variable names from previous or following years if necessary. For now I have just saved them without variable names.

For years 1992 to 1997, as well as 2002 onwards the dataset type changes from .tab to .por data. I have not yet figured out how to clean and read that, though the package pyreadstat is apparently able to open .por data. I suspect one of the authors of papers that makes use of this dataset might have a cleaner version of the dataset and/or an explanation for how they cleaned it, so I am going to email some of them. Most of these years also dont have a .dic file or a .txt file as in previous years containing all the variable names, questions, and corresponding answer scales. For years 2004 onwards the data changes again, and there are filetypes .sav, .dta, and .ovz that I have not figured out how to read either! I suspect one of them would contain the variable names but no clue which because I cannot open them!

In 2004 specifically (and I think one or two years after as well) they split the dataset into an a and b section:
P1692a: Respondents national sample
P1692b: Respondents disadvantaged neighbourhoods
So that's something to keep in mind as well.

Once all the data is cleaned and in one format I think a next step would be to add it all to one big file so we can see for what questions there is consistent year-by-year data.

In [320]:
import pandas as pd
import pickle 
import os

year = 1981 ## alter year to check different files

### FOR TAB DATA
# Read the .tab file
tab_file = [f for f in os.listdir(f'./{year}') if f.endswith('.tab')]
if len(tab_file) != 1:
    raise ValueError('should be only one tab file in the current directory')
print(tab_file[0])
data_number = tab_file[0].split(".")[0]

p1525a.tab


#### Old code

#### New Code

In [321]:
import re
from pathlib import Path

vardict = {}
scaledict = {}

current_var = None
collecting_values = False
value_lines = []

import re

def parse_dic_metadata(dic_path):
    with open(dic_path, "r", encoding="latin-1", errors="ignore") as f:
        lines = [line.rstrip("\n") for line in f]

    vardict = {}
    scaledict = {}
    current_var = None
    collecting_values = False
    value_lines = []

    # Regex patterns
    var_header = re.compile(r"^([A-Z]+\d+[A-Z]?)\s+.*?\s+(.+?)\s+\d+$")  # variable name + label
    value_line = re.compile(r"^\s*(-?\d+)\s*(?:[A-Z]+)?\s+(.*\S)")        # value line with optional letter code
    value_section_start = re.compile(r"\bValue\b.*\bLabel\b", re.IGNORECASE)

    for line in lines:
        line = line.strip()

        # Skip blank lines and page headers or file headers
        if not line or re.match(r'^\d{2} \w{3} \d{2}', line) or line.startswith("File:"):
            continue

        # Detect new variable
        m = var_header.match(line)
        if m:
            if current_var and value_lines:
                scaledict[current_var] = "\n".join(value_lines).strip()
                value_lines = []

            current_var = m.group(1).strip()
            vardict[current_var] = m.group(2).strip()
            collecting_values = False
            continue

        # Detect start of value section
        if value_section_start.search(line):
            collecting_values = True
            continue

        # Collect value lines
        if collecting_values:
            # Skip non-value lines inside the section
            if re.match(r'^(Print Format|Write Format|Missing Values):', line):
                continue

            # Stop collecting if next variable starts
            if var_header.match(line):
                collecting_values = False
                continue

            # Match value lines
            v = value_line.match(line)
            if v:
                val, lbl = v.groups()
                value_lines.append(f"{val} = {lbl.strip()}")
            else:
                # If line is continuation of previous label
                if value_lines:
                    value_lines[-1] += " " + line.strip()

    # Save last variable
    if current_var and value_lines:
        scaledict[current_var] = "\n".join(value_lines).strip()

    return vardict, scaledict



def parse_txt_metadata(txt_path):
    # Read the corresponding var names and scales for answers
    file1 = open(txt_path, "r")
    alltext = file1.read()
    varlabels = alltext.split("VAR LABELS")[1:-1]
    varlabels = " ".join(varlabels)
    valuelabels = alltext.split("VALUE LABELS")[1:-1]
    valuelabels = " ".join(valuelabels)
    file1.close()
    
    # Clean var names and scales text and turn into dictionary
    varnames = varlabels.split("/")
    varnames = [' '.join(item.split()) for item in varnames]
    vardict = {n.split(" ")[0]: " ".join(n.split(" ")[1:]) for n in varnames}

    varscales = valuelabels.split("/")
    varscales = [' '.join(item.split()) for item in varscales]
    scaledict = {n.split(" ")[0]: " ".join(n.split(" ")[1:]) for n in varscales}
    
    return vardict, scaledict



def find_dic(year):
    dat_dir = Path(str(year)) / "dat"
    if dat_dir.exists():
        dic_files = list(dat_dir.glob("*.dic"))
        if dic_files:
            return str(dic_files[0])   # return first .dic file

    year_dir = Path(str(year))
    if year_dir.exists():
        dic_files = list(year_dir.glob("*.dic"))
        if dic_files:
            return str(dic_files[0])
    return None

def find_txt(year):
    dat_dir = Path(str(year)) / "dat"
    if dat_dir.exists():
        dic_files = list(dat_dir.glob("p*.txt"))
        if dic_files:
            return str(dic_files[0])   # return first .dic file

    year_dir = Path(str(year))
    if year_dir.exists():
        dic_files = list(year_dir.glob("p*.txt"))
        if dic_files:
            return str(dic_files[0])
    return None

dic_path = find_dic(year)
txt_path = find_txt(year)

if dic_path:
    print("dic found")
    dic_path = f"{year}/dat/{data_number}.dic"
    with open(dic_path, "r", encoding="latin-1") as f:
        text = f.read()
    vardict, scaledict = parse_dic_metadata(dic_path)
elif txt_path:
    print("txt found")
    txt_path = f"{year}/dat/{data_number}.txt"
    vardict, scaledict = parse_txt_metadata(txt_path)
else:
    print("No .dic or .txt file found")

print("Variables parsed:", len(vardict))
print("Scales parsed:", len(scaledict))

#print(vardict["VAR003"])
#print(scaledict["VAR003"])

# Save variable name dictionary to separate folder clean_data
with open(f'clean_data/vardict_{year}.pkl', 'wb') as f:
    pickle.dump(vardict, f)
with open(f'clean_data/scaledict_{year}.pkl', 'wb') as f:
    pickle.dump(scaledict, f)

with open(f'clean_data/vardict_{year}.pkl', 'rb') as f:
    loaded_vardict = pickle.load(f)
with open(f'clean_data/scaledict_{year}.pkl', 'rb') as f:
    loaded_scaledict = pickle.load(f)


dic found
Variables parsed: 327
Scales parsed: 314


In [322]:
#print(vardict)
print(loaded_scaledict)

{'INT000': '1 = Compas\n2 = Capihome\n-7 = Uitgesloten\n-6 = Geen opgave\n-5 = N.v.t.\n-3 = Weet niet\n1 = Ja\n2 = Nee', 'INT001': '-7 = Uitgesloten 15:03:54  Steinmetz Archive              SUN SPARC        Solaris 2.3', 'INT004': '-7 = Uitgesloten\n-5 = N.v.t.', 'INT044': '-7 = Uitgesloten\n-6 = Geen opgave\n-3 = Weet niet\n1 = Sterk   eens\n2 = Mee eens\n3 = Noch,   noch\n4 = Mee     oneens\n5 = Sterk   oneens', 'INT071': '-7 = Uitgesloten\n-3 = Geen mening\n-2 = Wil niet zeggen\n1 = Volkomen eens\n2 = Gr lijn  eens\n3 = Noch,    noch\n4 = Eigenl   oneens\n5 = Helem    oneens', 'INT087': '-7 = Uitgesloten\n1 = Zonder partner\n2 = Bewust ongehuwd met\n3 = Met, later huwen\n4 = Gehuwd na proefper\n5 = Direct gehuwd 15:03:54  Steinmetz Archive              SUN SPARC        Solaris 2.3', 'INT229': '-7 = Uitgesloten\n-6 = Geen opgave\n-3 = Weet niet\n1 = Geen geloof\n2 = Onaantoonbaar\n3 = Wel hogere macht\n4 = Soms wel\n5 = Met twijfel\n6 = God bestaat!', 'INT239': '-7 = Uitgesloten\n-6 

In [323]:
# Keep variable IDs as columns
df = pd.read_csv(f'{year}/{tab_file[0]}', sep='\t')

# Build MultiIndex tuples
multi_columns = []
for var in df.columns:
    var_id = var
    var_question = loaded_vardict.get(var, "")
    var_scale = loaded_scaledict.get(var, "")
    multi_columns.append((var_id, var_question, var_scale))

# Create the MultiIndex
df.columns = pd.MultiIndex.from_tuples(multi_columns,
                                       names=["Variable", "Question", "Scale"])

df.head()

Variable,RESPNR,HHNR,HHPNR,INSTRU,INT000,INT001,INT004,INT044,INT071,INT087,...,VAR1162K,VAR1162L,VAR1162M,VAR1163,VAR1227,VAR1228,VAR1229,VAR1230,VAR3000,ISCO
Question,,,,,Schriftelijke deel ingevuld,Aantal jaar onderwijs vanaf 6e jaar,Aantal mensen waarover leiding,Overheid verkleinen verschil inkomens,Gezin lijdt onder vr+voll baan,Samenlevingsvorm,...,Oordeel: leefbaarheid steden,Oordeel: opvang buitenlanders,Oordeel: beleid kinderopvang,Nederl overh functioneert goed,Oorzaaken ziekte niet bestreden,Positief denken kan veel bereiken,Gebedsgenezers kunnen beter maken,"Negatief leven,leidt tot ziekte",CBS,
Scale,,,,,1 = Compas\n2 = Capihome\n-7 = Uitgesloten\n-6 = Geen opgave\n-5 = N.v.t.\n-3 = Weet niet\n1 = Ja\n2 = Nee,-7 = Uitgesloten 15:03:54 Steinmetz Archive SUN SPARC Solaris 2.3,-7 = Uitgesloten\n-5 = N.v.t.,"-7 = Uitgesloten\n-6 = Geen opgave\n-3 = Weet niet\n1 = Sterk eens\n2 = Mee eens\n3 = Noch, noch\n4 = Mee oneens\n5 = Sterk oneens","-7 = Uitgesloten\n-3 = Geen mening\n-2 = Wil niet zeggen\n1 = Volkomen eens\n2 = Gr lijn eens\n3 = Noch, noch\n4 = Eigenl oneens\n5 = Helem oneens","-7 = Uitgesloten\n1 = Zonder partner\n2 = Bewust ongehuwd met\n3 = Met, later huwen\n4 = Gehuwd na proefper\n5 = Direct gehuwd 15:03:54 Steinmetz Archive SUN SPARC Solaris 2.3",...,-7 = Uitgesloten\n-3 = Geen oordeel\n1 = Te veel\n2 = Juist genoeg\n3 = Te weinig 15:03:55 Steinmetz Archive SUN SPARC Solaris 2.3,-7 = Uitgesloten\n-3 = Geen oordeel\n1 = Te veel\n2 = Juist genoeg\n3 = Te weinig,-7 = Uitgesloten\n-3 = Geen oordeel\n1 = Te veel\n2 = Juist genoeg\n3 = Te weinig,-7 = Uitgesloten\n-3 = Geen mening\n1 = Zeer\n2 = Eens\n3 = Oneens\n4 = Zeer oneens,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar 15:03:55 Steinmetz Archive SUN SPARC Solaris 2.3,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar,18 = set listing 'p1525a.doc'.,
0,1,1,1,1,1,9,0,2,4,5,...,3,2,2,3,3,3,-3,-3,11108,5123
1,2,2,1,1,1,16,0,5,4,4,...,2,1,3,2,2,3,4,4,46708,7241
2,3,3,1,1,1,20,0,2,1,1,...,2,3,2,2,2,4,4,2,57210,3231
3,4,4,1,1,1,13,0,2,2,5,...,3,2,2,2,2,2,4,2,11117,8290
4,5,5,1,1,1,14,-5,2,2,5,...,2,3,3,2,2,4,3,2,-5,-5


In [324]:
# ---- FLATTEN MULTIINDEX FOR CSV OUTPUT ----
# Format: VAR003 | BURGERLIJKE STAAT | (1)... etc
df_flat = df.copy()
df_flat.columns = [' | '.join(filter(None, col)) for col in df.columns.to_list()]

# ---- SAVE TO CSV ----
output_path = f"clean_data/{year}.csv"
df_flat.to_csv(output_path, index=False)

print(f"Saved dataframe to {output_path}")

Saved dataframe to clean_data/2000.csv


In [325]:
for item in loaded_vardict:
    if "PROTEST" in loaded_vardict[item]:
        print(loaded_vardict[item])
    if "DEMONSTR" in loaded_vardict[item]:
        print(loaded_vardict[item])
    if "VRIJHEID" in loaded_vardict[item]:
        print(loaded_vardict[item])
    if "protest" in loaded_vardict[item]:
        print(loaded_vardict[item])
    if "demonstr" in loaded_vardict[item]:
        print(loaded_vardict[item])
    if "vrijheid" in loaded_vardict[item]:
        print(loaded_vardict[item])

Deelname demonstratie milieu afgel 5j
Vrij:om te demonstreren


In [326]:
df.head()

Variable,RESPNR,HHNR,HHPNR,INSTRU,INT000,INT001,INT004,INT044,INT071,INT087,...,VAR1162K,VAR1162L,VAR1162M,VAR1163,VAR1227,VAR1228,VAR1229,VAR1230,VAR3000,ISCO
Question,,,,,Schriftelijke deel ingevuld,Aantal jaar onderwijs vanaf 6e jaar,Aantal mensen waarover leiding,Overheid verkleinen verschil inkomens,Gezin lijdt onder vr+voll baan,Samenlevingsvorm,...,Oordeel: leefbaarheid steden,Oordeel: opvang buitenlanders,Oordeel: beleid kinderopvang,Nederl overh functioneert goed,Oorzaaken ziekte niet bestreden,Positief denken kan veel bereiken,Gebedsgenezers kunnen beter maken,"Negatief leven,leidt tot ziekte",CBS,
Scale,,,,,1 = Compas\n2 = Capihome\n-7 = Uitgesloten\n-6 = Geen opgave\n-5 = N.v.t.\n-3 = Weet niet\n1 = Ja\n2 = Nee,-7 = Uitgesloten 15:03:54 Steinmetz Archive SUN SPARC Solaris 2.3,-7 = Uitgesloten\n-5 = N.v.t.,"-7 = Uitgesloten\n-6 = Geen opgave\n-3 = Weet niet\n1 = Sterk eens\n2 = Mee eens\n3 = Noch, noch\n4 = Mee oneens\n5 = Sterk oneens","-7 = Uitgesloten\n-3 = Geen mening\n-2 = Wil niet zeggen\n1 = Volkomen eens\n2 = Gr lijn eens\n3 = Noch, noch\n4 = Eigenl oneens\n5 = Helem oneens","-7 = Uitgesloten\n1 = Zonder partner\n2 = Bewust ongehuwd met\n3 = Met, later huwen\n4 = Gehuwd na proefper\n5 = Direct gehuwd 15:03:54 Steinmetz Archive SUN SPARC Solaris 2.3",...,-7 = Uitgesloten\n-3 = Geen oordeel\n1 = Te veel\n2 = Juist genoeg\n3 = Te weinig 15:03:55 Steinmetz Archive SUN SPARC Solaris 2.3,-7 = Uitgesloten\n-3 = Geen oordeel\n1 = Te veel\n2 = Juist genoeg\n3 = Te weinig,-7 = Uitgesloten\n-3 = Geen oordeel\n1 = Te veel\n2 = Juist genoeg\n3 = Te weinig,-7 = Uitgesloten\n-3 = Geen mening\n1 = Zeer\n2 = Eens\n3 = Oneens\n4 = Zeer oneens,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar 15:03:55 Steinmetz Archive SUN SPARC Solaris 2.3,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar,-7 = Uitgesloten\n-3 = Weet niet\n1 = Absoluut waar\n2 = Waarschijnlijk waar\n3 = Waarschijnlijk niet\n4 = Absoluut niet waar,18 = set listing 'p1525a.doc'.,
0,1,1,1,1,1,9,0,2,4,5,...,3,2,2,3,3,3,-3,-3,11108,5123
1,2,2,1,1,1,16,0,5,4,4,...,2,1,3,2,2,3,4,4,46708,7241
2,3,3,1,1,1,20,0,2,1,1,...,2,3,2,2,2,4,4,2,57210,3231
3,4,4,1,1,1,13,0,2,2,5,...,3,2,2,2,2,2,4,2,11117,8290
4,5,5,1,1,1,14,-5,2,2,5,...,2,3,3,2,2,4,3,2,-5,-5
